In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install fanalysis

from sklearn.metrics import confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# les noms des variables

colonnes =(["IDENT", "TypeHabi", "GeoFaz_Scl", "GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Anciennete",
            "Logement_prix_m2", "Mediane_RFR", "Sexe", "Prenom_Phase_scl_H", "Prenom_Phase_scl_F",
            "Prenom_Phase_cl_H", "Prenom_Phase_cl_F", "Age_H", "Age_F", "Affinite_1", "Affinite_2",
            "Prenom_effectif", "Potentiel_prenom", "Onoma-Phase_Scl", "Onoma-Phase_CL", "Top_mon"])

In [3]:
# importation du fichier (changer le chemin d'accès)
''' Faites attention au type de fichier, au délimiteur des valeurs (ça doit être unique et non multiple),
au header et à la lecture des types d'objets (low_memory) '''

data_orig = pd.read_csv("/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/my_data_FDFTOT21.txt", delim_whitespace="\t", header = None, names=colonnes, low_memory=False, )#, dtype='Int64')


In [4]:
''' copie superficielle (peu profonde) du dataset original pour ne pas modifier les valeurs
d'origine par erreur plus tard 
'''
data = data_orig.copy()
data.shape

(1219999, 23)

In [5]:
# visualisation des premiers lignes
data.head()

,IDENT,TypeHabi,GeoFaz_Scl,GeoFaz_cl,ImmoFaz_Scl,ImmoFaz_cl,Anciennete,Logement_prix_m2,Mediane_RFR,Sexe,Prenom_Phase_scl_H,Prenom_Phase_scl_F,Prenom_Phase_cl_H,Prenom_Phase_cl_F,Age_H,Age_F,Affinite_1,Affinite_2,Prenom_effectif,Potentiel_prenom,Onoma-Phase_Scl,Onoma-Phase_CL,Top_mon
0,10010000001,2,81,8,91,9,6,21,21,1,71,41,7,4,12,4,20,2,2,1,51,5,1.0
1,20010000006,1,75,7,43,4,6,6,13,2,71,64,7,6,12,9,4,12,4,13,12,1,1.0
2,30010000011,1,76,7,43,4,6,6,16,3,54,81,5,8,7,12,6,6,4,18,36,3,2.0
3,40010000013,3,75,7,43,4,4,6,13,1,71,36,7,3,12,11,11,19,4,6,21,2,1.0
4,50010000015,2,33,3,44,4,6,7,12,1,71,42,7,4,12,5,14,3,3,1,14,1,2.0


In [6]:
#nombre de modalités de chaque variables (dans un dataframe)
def nombre_mod(data2):
    #création d'un dataframe vide
    dff= pd.DataFrame(columns=["VARIABLE","NOMBRE_MOD", "NOMBRE_MOD_SANS_NA"])
    for i in range(data2.shape[1]): # sans compter les valeurs manquantes
        taille =len(data2.iloc[:,i].value_counts(dropna=False))
        taille_1 = len(data2.iloc[:,i].value_counts(dropna=True) )
        dff=dff.append({"VARIABLE": colonnes[i],"NOMBRE_MOD":taille, "NOMBRE_MOD_SANS_NA":taille_1 }, ignore_index=True)
    return dff

# sélection des corrélations les plus importantes
def variables_corr(tableau_corr, seuil):
    colonnes = tableau_corr.columns
    val_temp = 0 
    variables_corr_pos=[] ; variables_corr_pos_val=[]
    variables_corr_neg=[] ; variables_corr_neg_val=[]
    for i in range(tableau_corr.shape[0]):  
        for j in range(tableau_corr.shape[0]):
            if i<j: #i!=j:
                if (tableau_corr.iloc[i,j] > seuil):
                    variables_corr_pos.append(colonnes[j])
                    variables_corr_pos_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

                if (tableau_corr.iloc[i,j] <-seuil):
                    variables_corr_neg.append(colonnes[j])
                    variables_corr_neg_val.append(round( tableau_corr.iloc[i,j], 2))
                    val_temp += 1

        if (len(variables_corr_pos) > 0) | (len(variables_corr_neg) >0) :
            print("variable : ", colonnes[i])
            if len(variables_corr_pos) > 0:    
                print("corr pos", variables_corr_pos)
                print("corr pos", variables_corr_pos_val)
                val_temp += 1

            if len(variables_corr_neg) >0:  
                print("corr neg", variables_corr_neg)
                print("corr neg", variables_corr_neg_val)
                val_temp += 1
            print("")
        variables_corr_pos = variables_corr_pos*0
        variables_corr_neg = variables_corr_pos*0

        variables_corr_pos_val = variables_corr_pos_val*0
        variables_corr_neg_val = variables_corr_pos_val*0
    if val_temp == 0:
        print("IL N'Y A PAS DE CORRELATION ENTRE LES VARIABLES PAR RAPPORT AU SEUIL PRIS")
        

def evaluation(model, X_train, y_train, X_test, y_test, scoring, plot):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print("test set : ")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred)) 
    print("train set : ")
    print(confusion_matrix(y_train, model.predict(X_train)))  
    print(classification_report(y_train, model.predict(X_train) ))

    if plot == True:
        N, train_score, val_score = learning_curve(model, X_train, y_train, shuffle=True, cv = 4, scoring = scoring, train_sizes = np.linspace(0.1, 1, 6) )

        plt.figure(figsize = (10,6))
        plt.plot(N, train_score.mean(axis=1), label ="train_score")
        plt.plot(N, val_score.mean(axis=1), label ="cross_validation_score")
        plt.legend()
    else :
        print("GRAPHIQUE NON DEMANDE")

# Comme il n'y a pas de valeurs manquantes (à la place il y a un nombre), le seul traitement qu'on fait c'est tranformer
# le type de variable (les variables continues deviennent qualitatives)
def data_processing(df):
    """ 
    for i in range(0,len(df.columns)):    
        df[df.columns[i]].fillna(100.0, inplace=True) #On remplace les données manquantes par 100
    for i in range(0,len(df.columns)):
        print(df[df.columns[i]].value_counts(dropna=False))    
    """
    
   #Transformation de toutes les variables en facteurs         
    for i in range(0,len(df.columns)):
        df[df.columns[i]] = df[df.columns[i]].astype('category')
    return df.to_numpy() #.as_matrix()


def vingtiles(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    #print(df_tous)
    
    # pour la colonne Top_mon :
    # remplissage de la dernière colonne par 2 (VALABLE QUE POUR CE DATA, MODIFIER POUR D'AUTRES)
    data_temp_t = data_orig.copy()
    # remplacement
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(1.0,'1')
    data_temp_t["Top_mon"] =  data_temp_t["Top_mon"].replace(2.0,'2')
    data_all_complet_class = data_temp_t.fillna('2')
    #data_all_complet_class

    # jointure
    ident_class_tous = data_all_complet_class[["IDENT","Top_mon"]]
    tableau_resultats = pd.merge(df_tous,ident_class_tous)
    #tableau_resultats

    # tranches finales
    tab_cross_temp = pd.crosstab(tableau_resultats["TRANCHES"],tableau_resultats["Top_mon"])
    #print(tab_cross_temp); print("-----------------------")

    # création d'une colonne total
    tab_cross_temp["TOTAL"]=tab_cross_temp["1"]+tab_cross_temp["2"]
    
    # affichage
    if affichage ==True:        
        print(tab_cross_temp)

    # création du tableau final en éliminant la colonne 2
    #tab_pour_excel = tab_cross_temp.drop(["2"], axis=1)

    # si on ne veut pas éliminer la colonne 2
    tab_pour_excel =  tab_cross_temp
    
    # enregistrement sous Excel
    if save == True:
        tab_pour_excel.to_excel(path)

def vingtiles_ind_sans_target(model, data_orig, affichage, save, path):
    # position de la dernière variable explicative dans le dataset
    nb_var = data_orig.shape[1]-1

    # predict sur tous les individus (avec et sans classe IFI+)
    tous_probas = model.predict_proba(data_orig.iloc[:,1:nb_var])
    df_tous = pd.DataFrame({'IDENT': data_orig.iloc[:,0] , 'PROBA': tous_probas[:,0] })
    #df_tous
    
    # valeurs ordonnées pour diviser après
    df_tous=df_tous.sort_values(by='PROBA', ascending=True)
    #df_tous
    print(df_tous)
    
    #vérification
    print(df_tous['PROBA'].value_counts())

    # quarantiles
    df_temp =pd.DataFrame( pd.qcut(df_tous['PROBA'],q= 20)  )
    print(df_temp)

    # ajout par index (IMPORTANTE)
    df_tous["TRANCHES"]=pd.DataFrame(df_temp.iloc[:,0].to_numpy(),index= df_temp.index)
    print(df_tous)
    print(df_tous["TRANCHES"].value_counts(ascending = True))

    # GERE L'ORDRE
    patra = np.unique(df_tous.iloc[:,2], return_counts=True)
    df_tranches_sans_target = pd.DataFrame([patra[0],patra[1]]).T
    df_tranches_sans_target.columns=["TRANCHES","COUNTS"]

    # affichage
    if affichage ==True:        
        print(df_tranches_sans_target)
    
    # enregistrement sous Excel
    if save == True:
        df_tranches_sans_target.to_excel(path)
    

# élimination des variables trop corrélées (choix manuel)
def elimination_val_corr(data):
    data = data.drop(columns=[""])
    return data
          
        


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve, GridSearchCV

In [ ]:
####################################################################################
#                            MODELE 1 : BOOSTING
####################################################################################

In [ ]:
####################   SANS LES VARIABLES CORR ET ENCODAGE DUMMIES        ###########################

In [8]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F","Onoma-Phase_Scl",
                "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe IFI+
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
model_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight = "balanced"), random_state =0 )
param = {'base_estimator__criterion': ['gini','entropy'], 'base_estimator__max_depth' : list(range(2,20, 2)), 'n_estimators' : [20, 50] }

grid_AdaBoost = GridSearchCV(model_AdaBoost, param, cv = 4, scoring = "recall")
resultat_grid_AdaBoost = grid_AdaBoost.fit(train_X_s, train_Y_s)
resultat_grid_AdaBoost.best_params_
#model_AdaBoost.get_params


{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'n_estimators': 50}

In [9]:
final_model_AdaBoost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(class_weight="balanced", criterion="entropy",max_depth=2),n_estimators = 50 , random_state=0 ) 
evaluation(final_model_AdaBoost, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall" , plot =False)

test set : 
[[ 2500  2179]
 [ 7424 13320]]
              precision    recall  f1-score   support

           1       0.25      0.53      0.34      4679
           2       0.86      0.64      0.74     20744

    accuracy                           0.62     25423
   macro avg       0.56      0.59      0.54     25423
weighted avg       0.75      0.62      0.66     25423

train set : 
[[10335  8442]
 [28817 54098]]
              precision    recall  f1-score   support

           1       0.26      0.55      0.36     18777
           2       0.87      0.65      0.74     82915

    accuracy                           0.63    101692
   macro avg       0.56      0.60      0.55    101692
weighted avg       0.75      0.63      0.67    101692

GRAPHIQUE NON DEMANDE


In [13]:
# TRAIN SET TEST SET
vingtiles(model = final_model_AdaBoost, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Adaboost.xlsx")

                   IDENT     PROBA
42850    428510010357636  0.188269
42851    428520010357636  0.188269
46649    466500010360721  0.188479
52456    524570010365731  0.189056
38876    388770010350261  0.319804
...                  ...       ...
40877    408780010353089  0.671213
99873    998740010413228  0.671858
13620    136210010257604  0.672557
107013  1070140010502776  0.674065
111901  1119020010521403  0.676399

[127115 rows x 2 columns]
0.496489    515
0.500127    352
0.497421    275
0.499830    240
0.496207    236
           ... 
0.493898      1
0.494560      1
0.499133      1
0.500211      1
0.496313      1
Name: PROBA, Length: 54952, dtype: int64
                   PROBA
42850   (0.1882, 0.4959]
42851   (0.1882, 0.4959]
46649   (0.1882, 0.4959]
52456   (0.1882, 0.4959]
38876   (0.1882, 0.4959]
...                  ...
40877   (0.5045, 0.6764]
99873   (0.5045, 0.6764]
13620   (0.5045, 0.6764]
107013  (0.5045, 0.6764]
111901  (0.5045, 0.6764]

[127115 rows x 1 columns]
Top_mon  

In [14]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_AdaBoost, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Adaboost.xlsx")

                   IDENT     PROBA
697350  5702360270002309  0.185851
523877  3967630190012322  0.188269
199093   719790050011969  0.188269
374176  2470620120044331  0.188404
199176   720620050012052  0.188479
...                  ...       ...
845045  7179310290004999  0.674267
790884  6637700280075843  0.674267
259411  1322970060002288  0.674309
778120  6510060280063079  0.675774
160223   331090020023107  0.676399

[1092884 rows x 2 columns]
0.496489    5889
0.500127    2982
0.497421    2844
0.496207    2705
0.497110    2411
            ... 
0.502277       1
0.499980       1
0.503141       1
0.494205       1
0.500590       1
Name: PROBA, Length: 277159, dtype: int64
                                PROBA
697350  (0.18580000000000002, 0.4959]
523877  (0.18580000000000002, 0.4959]
199093  (0.18580000000000002, 0.4959]
374176  (0.18580000000000002, 0.4959]
199176  (0.18580000000000002, 0.4959]
...                               ...
845045               (0.5054, 0.6764]
790884             

In [ ]:
#############################################################
#                   MODELE 2 : RANDOM FOREST
#############################################################

In [ ]:
model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", random_state =0 )
param = {'criterion': ['gini','entropy'], 'max_depth' : [2,3,4,5],"max_features":[8,12,15,20,25,30] ,'n_estimators' : [20,50,100,200] }

grid_RandomForestClassifier = GridSearchCV(model_RandomForestClassifier, param, cv = 4, scoring = "recall", n_jobs=2)
resultat_grid_RandomForestClassifier = grid_RandomForestClassifier.fit(train_X_s, train_Y_s)
resultat_grid_RandomForestClassifier.best_params_


{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 25,
 'n_estimators': 100}

In [15]:
final_model_RandomForestClassifier = RandomForestClassifier(class_weight="balanced", criterion="entropy", max_depth=2, max_features=25, n_estimators = 100, random_state=0) 
evaluation(final_model_RandomForestClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 2941  1738]
 [10016 10728]]
              precision    recall  f1-score   support

           1       0.23      0.63      0.33      4679
           2       0.86      0.52      0.65     20744

    accuracy                           0.54     25423
   macro avg       0.54      0.57      0.49     25423
weighted avg       0.74      0.54      0.59     25423

train set : 
[[11814  6963]
 [39906 43009]]
              precision    recall  f1-score   support

           1       0.23      0.63      0.34     18777
           2       0.86      0.52      0.65     82915

    accuracy                           0.54    101692
   macro avg       0.54      0.57      0.49    101692
weighted avg       0.74      0.54      0.59    101692

GRAPHIQUE NON DEMANDE


In [16]:
# TRAIN SET TEST SET
vingtiles(model = final_model_RandomForestClassifier, data_orig = data, affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Random_Forest.xlsx")

                   IDENT     PROBA
43772    437730010358349  0.440510
31342    313430010341310  0.441143
33170    331710010343385  0.441367
82750    827510010397541  0.441367
26831    268320010337388  0.441669
...                  ...       ...
70696    706970010380915  0.674459
103441  1034420010416386  0.674459
98139    981400010411310  0.674459
43088    430890010357839  0.674459
62409    624100010373476  0.674459

[127115 rows x 2 columns]
0.460333    6134
0.512956    3196
0.507179    2098
0.460280    1994
0.511672    1462
            ... 
0.554860       1
0.532087       1
0.475308       1
0.504877       1
0.496743       1
Name: PROBA, Length: 12404, dtype: int64
                 PROBA
43772    (0.44, 0.449]
31342    (0.44, 0.449]
33170    (0.44, 0.449]
82750    (0.44, 0.449]
26831    (0.44, 0.449]
...                ...
70696   (0.558, 0.674]
103441  (0.558, 0.674]
98139   (0.558, 0.674]
43088   (0.558, 0.674]
62409   (0.558, 0.674]

[127115 rows x 1 columns]
Top_mon            1  

In [17]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_RandomForestClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=True, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Random_Forest.xlsx")

                     IDENT     PROBA
804064    6769500280089023  0.440025
284484    1573700080000656  0.440025
806878    6797640280091837  0.440025
610707    4835930220025126  0.440510
637886    5107720230002864  0.440510
...                    ...       ...
1177925  10508110370067274  0.673465
657368    5302540250002333  0.673465
1145278  10181640370034627  0.673465
1209981  10828670370099330  0.674459
1180713  10535990370070062  0.674459

[1092884 rows x 2 columns]
0.460333    56613
0.512956    33789
0.460280    16712
0.511672    15412
0.469914    14246
            ...  
0.522698        1
0.488675        1
0.519637        1
0.569254        1
0.477253        1
Name: PROBA, Length: 36105, dtype: int64
                      PROBA
804064   (0.44001, 0.44838]
284484   (0.44001, 0.44838]
806878   (0.44001, 0.44838]
610707   (0.44001, 0.44838]
637886   (0.44001, 0.44838]
...                     ...
1177925  (0.56207, 0.67446]
657368   (0.56207, 0.67446]
1145278  (0.56207, 0.67446]
1209981  

In [ ]:
###################################################################################################################@
###################################################################################################################@
#                                   DECISION TREE CLASSIFIER  ----  AVEC VARIABLES QUALITATIVES
###################################################################################################################@
###################################################################################################################@

In [ ]:
####################################################################################
####################################################################################
#                   TEST SANS LES VARIABLES ILLUSTRATIVES DE SPAD
####################################################################################
####################################################################################

In [ ]:
# test predict individus sans classe
data = data_orig.copy()

# on élimine les variables dons
data=data.drop(["GeoFaz_cl", "ImmoFaz_Scl", "ImmoFaz_cl", "Prenom_Phase_cl_H", "Prenom_Phase_cl_F",
                "Onoma-Phase_Scl", "Onoma-Phase_CL"],axis=1)

# encodage Onehot sur toutes les données (on évite ainsi les erreurs futures)
nb_var = data.shape[1] - 1
data_1_22 = pd.get_dummies(data.iloc[:,1:(data.shape[1]-1)], columns=data.columns[1:(data.shape[1]-1)] )
data_var_qualitatives_temp =  pd.concat([data["IDENT"], data_1_22, data["Top_mon"]], axis=1)
data = data_var_qualitatives_temp
#encodage = OneHotEncoder()
#encodage.fit(data_processing(data.iloc[:,1:nb_var]))

# élimination des individus n'ayant pas de valeur pour la varialbe IFI+
"""IMPORTANT"""
data_sans_nan = data[(data.isnull()).sum(axis=1) == 0]

#conversion du type de variable
for i in range(1,data_sans_nan.shape[1]):
    data_sans_nan.iloc[:,i]=data_sans_nan.iloc[:,i].astype(int)

# base de données avec des variables corrélées et avec encodage OneHot
data =data_sans_nan

nb_var = data.shape[1] - 1
train_set, test_set = train_test_split(data, test_size = 0.2, random_state=0 )

train_X_s = (train_set.iloc[:,1:nb_var]).to_numpy()
train_Y_s = np.asanyarray(train_set.iloc[:,nb_var])

test_X_s = (test_set.iloc[:,1:nb_var]).to_numpy()
test_Y_s = np.asanyarray(test_set.iloc[:,nb_var])


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:

model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, class_weight = "balanced" )
param = { 'max_depth' :list(range(2,30)), 'criterion': ["gini", "entropy"], 'max_features' : list(range(5,30,2)),
         'min_samples_split' : [0.05, 0.1, 0.4, 0.6, 0.8, 1] }

grid_DecisionTreeClassifier = GridSearchCV(model_DecisionTreeClassifier, param_grid= param, cv = 4, scoring = "recall", n_jobs=2)
resultat_grid_DecisionTreeClassifier = grid_DecisionTreeClassifier.fit(train_X_s, train_Y_s)
resultat_grid_DecisionTreeClassifier.best_params_


{'criterion': 'gini',
 'max_depth': 2,
 'max_features': 13,
 'min_samples_split': 0.05}

In [18]:
final_model_DecisionTreeClassifier = DecisionTreeClassifier(random_state = 0, criterion="gini", class_weight = "balanced", max_depth = 2, max_features=13, min_samples_split=0.05 )
evaluation(final_model_DecisionTreeClassifier, train_X_s, train_Y_s, test_X_s, test_Y_s, scoring="recall", plot=False)

test set : 
[[ 4552   127]
 [19920   824]]
              precision    recall  f1-score   support

           1       0.19      0.97      0.31      4679
           2       0.87      0.04      0.08     20744

    accuracy                           0.21     25423
   macro avg       0.53      0.51      0.19     25423
weighted avg       0.74      0.21      0.12     25423

train set : 
[[18270   507]
 [79493  3422]]
              precision    recall  f1-score   support

           1       0.19      0.97      0.31     18777
           2       0.87      0.04      0.08     82915

    accuracy                           0.21    101692
   macro avg       0.53      0.51      0.20    101692
weighted avg       0.74      0.21      0.12    101692

GRAPHIQUE NON DEMANDE


In [19]:
# TRAIN SET TEST SET
vingtiles(model = final_model_DecisionTreeClassifier, data_orig = data, affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/train_test_Decision_Tree_Classifier.xlsx")

                   IDENT     PROBA
83556    835570010398180  0.395492
49075    490760010362694  0.395492
5803      58040010060815  0.395492
29796    297970010339741  0.395492
84546    845470010398971  0.395492
...                  ...       ...
25866    258670010335476  0.544734
111904  1119050010521405  0.544734
67526    675270010377572  0.544734
29122    291230010339193  0.544734
85404    854050010399675  0.544734

[127115 rows x 2 columns]
0.501427    116139
0.544734      6096
0.395492      4880
Name: PROBA, dtype: int64


ValueError: ignored

In [22]:
# INDIVIDUS SANS TARGET
vingtiles_ind_sans_target(model = final_model_DecisionTreeClassifier, data_orig= data_var_qualitatives_temp[(data_var_qualitatives_temp.isnull()).sum(axis=1) != 0], affichage=True,
            save=False, path = "/content/drive/MyDrive/spad_score_method_1/spad_projection_FDFTOT21/Fichier_excel_vingtiles/ind_sans_target_Decision_Tree_Classifier.xlsx")

                     IDENT     PROBA
400010    2728960130020175  0.395492
346069    2189550120016224  0.395492
205261     781470050018137  0.395492
346064    2189500120016219  0.395492
1141792  10146780370031141  0.395492
...                    ...       ...
370446    2433320120040601  0.544734
180692     535780040003576  0.544734
1097940   9708260360147289  0.544734
180747     536330040003631  0.544734
1141212  10140980370030561  0.544734

[1092884 rows x 2 columns]
0.501427    997647
0.544734     51129
0.395492     44108
Name: PROBA, dtype: int64


ValueError: ignored

In [ ]:
train_X_s.shape

(101692, 374)

In [ ]:
np.sqrt(374)

19.339079605813716

In [ ]:
np.linspace(0.1, 1,5)

array([0.1  , 0.325, 0.55 , 0.775, 1.   ])

In [ ]:
0.05, 0.1, 0.4, 0.6, 0.8, 1